# ショット切り出し  

以下の処理を行います。  

* ショット切り出し
* 中断区間のデータ除外
* 物理変換 + 校正
* SPM計算
* 事象記録のタグ付け
* SPMから算出される、ショット当たりの最大サンプル数を超えたショット除外
* Elasticsearchインデックスへの保存
  * shots-yyyyMMddHHMMSS-data：切り出されたショットデータ
  * shots-yyyyMMddHHMMSS-meta：ショットのメタデータ
  

In [1]:
# ライブラリインポート
import os, sys
sys.path.append(os.path.join(os.getcwd(), '../src/'))
from cut_out_shot.cut_out_shot import CutOutShot
from utils import common
import logging

In [2]:
# ログ設定
LOG_FILE = os.path.join(
    common.get_config_value(common.APP_CONFIG_PATH, "log_dir"), "cut_out_shot/cut_out_shot.log"
)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.handlers.RotatingFileHandler(
            LOG_FILE, maxBytes=common.MAX_LOG_SIZE, backupCount=common.BACKUP_COUNT
        ),
        logging.StreamHandler(),
    ],
)

### CutOutShot クラスパラメーター一覧  

**【必須】**  
***displacement_func***  
変位センサーの物理変換式  

***load01_func***  
荷重センサー01の物理変換式  

***load02_func***  
荷重センサー02の物理変換式  

***load03_func***  
荷重センサー03の物理変換式  

***load04_func***  
荷重センサー04の物理変換式  


**【任意】**  
***min_spm***  
最小SPM。この値をもとに、1ショットに含まれる最大サンプル数を計算し、最大サンプル数を超えたショットは切り出し対象外とする。  
指定しない場合、既定値の15となる。  

***back_seconds_for_tagging***  
データ収集マネージャーで記録した事象について、何秒遡ってタグ付けするかを指定。指定しない場合、既定値の120となる。

***previous_size***  
ショットを検知したサンプルから何サンプル遡ってショットに含めるかを指定。これは切り出しショットに荷重開始点が確実に含まれるようにするため。指定しない場合、既定値の1000となる。


### cut_out_shot 関数パラメーター一覧
**【必須】**  
***rawdata_dir_name***  
ショット切り出しの対象とする生データファイルのディレクトリ（.pklファイルが含まれるディレクトリ）。  
ディレクトリ名はデータ収集開始日時で、yyyyMMddHHMMSSの形式となっている。

***start_displacement***  
ショット切り出しの開始を検知するための変位値。  

***end_displacement***  
ショット切り出しの終了を検知するための変位値。  

**【任意】**  
***start_sequential_number***  
ショット切り出しの開始連番。指定しない場合、データの先頭から切り出し。  

***end_sequential_number***  
ショット切り出しの終了連番。指定しない場合、データの末尾まで切り出し。  

In [3]:
# データディレクトリの一覧
!ls /datadrive/data

ls: cannot access '/datadrive/data': No such file or directory


In [4]:
# 物理変換式設定
displacement_func = lambda v: 70.0 - (v - 2.0) * 70.0 / 8.0
Vr = 2.5
load01_func = lambda v: 2.5 / Vr * v
load02_func = lambda v: 2.5 / Vr * v
load03_func = lambda v: 2.5 / Vr * v
load04_func = lambda v: 2.5 / Vr * v

In [5]:
# 適宜変更してください
rawdata_dir_name="20210327141514"
start_displacement=47.0
end_displacement=34.0
# start_sequential_number=20000
# end_sequential_number=140000

cut_out_shot = CutOutShot(
    displacement_func=displacement_func,
    load01_func=load01_func,
    load02_func=load02_func,
    load03_func=load03_func,
    load04_func=load04_func,
    margin=0.3,
    previous_size=1000
)

cut_out_shot.cut_out_shot(
    rawdata_dir_name=rawdata_dir_name, 
    start_displacement=start_displacement, 
    end_displacement=end_displacement,
#     start_sequential_number=start_sequential_number,
#     end_sequential_number=end_sequential_number
)

2021-04-01 08:22:38,269 [INFO] Cut out shot start.
2021-04-01 08:22:38,351 [INFO] delete index 'shots-20210327141514-data' finished. result: {'acknowledged': True}
2021-04-01 08:22:38,541 [INFO] create index 'shots-20210327141514-data' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'shots-20210327141514-data'}
2021-04-01 08:22:38,598 [INFO] delete index 'shots-20210327141514-meta' finished. result: {'acknowledged': True}
2021-04-01 08:22:38,787 [INFO] create index 'shots-20210327141514-meta' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'shots-20210327141514-meta'}
2021-04-01 08:22:38,977 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-051514.339398.pkl
2021-04-01 08:22:39,239 [INFO] Thoughput: 304759 doc/sec, processed_count: 137216
2021-04-01 08:22:39,240 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/2

2021-04-01 08:22:47,732 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-051601.200211.pkl
2021-04-01 08:22:48,028 [INFO] Thoughput: 524280 doc/sec, processed_count: 4843520
2021-04-01 08:22:48,029 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-051602.771208.pkl
2021-04-01 08:22:48,318 [INFO] Thoughput: 524719 doc/sec, processed_count: 5000192
2021-04-01 08:22:48,319 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-051604.343948.pkl
2021-04-01 08:22:48,610 [INFO] Thoughput: 525196 doc/sec, processed_count: 5157888
2021-04-01 08:22:48,611 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-051605.915432.pkl
2021-04-01 08:22:48,904 [INFO] Thoughput: 525543 doc/sec, processed_cou

2021-04-01 08:23:00,264 [INFO] Thoughput: 459764 doc/sec, processed_count: 9873408
2021-04-01 08:23:00,265 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-051653.076252.pkl
2021-04-01 08:23:00,579 [INFO] Thoughput: 460350 doc/sec, processed_count: 10031104
2021-04-01 08:23:00,722 [INFO] 9877 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-051654.648719.pkl.
2021-04-01 08:23:01,102 [INFO] Thoughput: 456641 doc/sec, processed_count: 10188800
2021-04-01 08:23:01,103 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-051656.222950.pkl
2021-04-01 08:23:01,423 [INFO] Thoughput: 457071 doc/sec, processed_count: 10345472
2021-04-01 08:23:01,547 [INFO] 8640 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-0516

2021-04-01 08:23:12,343 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-051743.397615.pkl
2021-04-01 08:23:12,631 [INFO] Thoughput: 445105 doc/sec, processed_count: 15063040
2021-04-01 08:23:12,632 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-051744.970559.pkl
2021-04-01 08:23:12,921 [INFO] Thoughput: 445934 doc/sec, processed_count: 15220736
2021-04-01 08:23:12,922 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-051746.541519.pkl
2021-04-01 08:23:13,213 [INFO] Thoughput: 446735 doc/sec, processed_count: 15378432
2021-04-01 08:23:13,214 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-051748.117823.pkl
2021-04-01 08:23:13,499 [INFO] Thoughput: 447571 doc/sec, processed_

2021-04-01 08:23:21,961 [INFO] Thoughput: 465444 doc/sec, processed_count: 20093952
2021-04-01 08:23:22,074 [INFO] 7653 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-051835.283003.pkl.
2021-04-01 08:23:22,490 [INFO] Thoughput: 463412 doc/sec, processed_count: 20251648
2021-04-01 08:23:22,544 [INFO] 3203 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-051836.851357.pkl.
2021-04-01 08:23:22,961 [INFO] Thoughput: 462022 doc/sec, processed_count: 20408320
2021-04-01 08:23:23,110 [INFO] 10525 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-051838.424322.pkl.
2021-04-01 08:23:23,497 [INFO] Thoughput: 460010 doc/sec, processed_count: 20566016
2021-04-01 08:23:23,498 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-0

...

2021-04-01 08:46:11,506 [INFO] 7673 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064546.529294.pkl.
2021-04-01 08:46:11,928 [INFO] Thoughput: 357732 doc/sec, processed_count: 505526271
2021-04-01 08:46:12,036 [INFO] 7669 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064548.107798.pkl.
2021-04-01 08:46:12,508 [INFO] Thoughput: 357697 doc/sec, processed_count: 505683967
2021-04-01 08:46:12,656 [INFO] 10541 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064549.686340.pkl.
2021-04-01 08:46:13,103 [INFO] Thoughput: 357659 doc/sec, processed_count: 505842687
2021-04-01 08:46:13,225 [INFO] 8615 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064551.267853.pkl.
2021-04-01 08:46:14,130 [INFO] Thoughput: 357511 doc/sec, proces

2021-04-01 08:46:31,285 [INFO] Thoughput: 356426 doc/sec, processed_count: 510579711
2021-04-01 08:46:31,394 [INFO] 7683 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064638.644507.pkl.
2021-04-01 08:46:31,844 [INFO] Thoughput: 356398 doc/sec, processed_count: 510738431
2021-04-01 08:46:31,992 [INFO] 7678 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064640.223167.pkl.
2021-04-01 08:46:32,421 [INFO] Thoughput: 356364 doc/sec, processed_count: 510896127
2021-04-01 08:46:32,537 [INFO] 7698 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064641.805594.pkl.
2021-04-01 08:46:32,969 [INFO] Thoughput: 356338 doc/sec, processed_count: 511053823
2021-04-01 08:46:33,079 [INFO] 7702 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327

2021-04-01 08:46:49,273 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064729.195072.pkl
2021-04-01 08:46:49,570 [INFO] Thoughput: 355527 doc/sec, processed_count: 515792895
2021-04-01 08:46:49,571 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064730.776643.pkl
2021-04-01 08:46:49,882 [INFO] Thoughput: 355559 doc/sec, processed_count: 515950591
2021-04-01 08:46:49,883 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064732.354068.pkl
2021-04-01 08:46:50,186 [INFO] Thoughput: 355593 doc/sec, processed_count: 516107263
2021-04-01 08:46:50,187 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064733.919192.pkl
2021-04-01 08:46:50,484 [INFO] Thoughput: 355629 doc/sec, process

2021-04-01 08:47:03,793 [INFO] Thoughput: 355523 doc/sec, processed_count: 520843263
2021-04-01 08:47:03,941 [INFO] 10431 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064821.279140.pkl.
2021-04-01 08:47:04,335 [INFO] Thoughput: 355500 doc/sec, processed_count: 521001983
2021-04-01 08:47:04,336 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064822.859008.pkl
2021-04-01 08:47:04,687 [INFO] Thoughput: 355522 doc/sec, processed_count: 521159679
2021-04-01 08:47:04,841 [INFO] 10483 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064824.441746.pkl.
2021-04-01 08:47:05,278 [INFO] Thoughput: 355486 doc/sec, processed_count: 521317375
2021-04-01 08:47:05,425 [INFO] 10443 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210

2021-04-01 08:47:15,603 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064911.858038.pkl
2021-04-01 08:47:15,913 [INFO] Thoughput: 356137 doc/sec, processed_count: 526059519
2021-04-01 08:47:15,914 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064913.440956.pkl
2021-04-01 08:47:16,234 [INFO] Thoughput: 356167 doc/sec, processed_count: 526217215
2021-04-01 08:47:16,234 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064915.020710.pkl
2021-04-01 08:47:16,583 [INFO] Thoughput: 356189 doc/sec, processed_count: 526374911
2021-04-01 08:47:16,585 [INFO] Shot is not detected in /home/ymiyamoto5/h-one-experimental-system/shared/data/20210327141514/GW-01_AD-USB-1608_20210327-064916.597646.pkl
2021-04-01 08:47:16,889 [INFO] Thoughput: 356223 doc/sec, process

<br>
<br>
<br>
<br>
<br>
<br>
<hr>

## 参考）テストデータでのショット切り出し

In [9]:
# 適宜変更してください
rawdata_dir_name="20201201010000"
start_displacement=47.0
end_displacement=34.0
start_sequential_number=None
end_sequential_number=None

displacement_func = lambda v: v
load01_func = lambda v: v
load02_func = lambda v: v
load03_func = lambda v: v
load04_func = lambda v: v

cut_out_shot = CutOutShot(
    displacement_func=displacement_func,
    load01_func=load01_func,
    load02_func=load02_func,
    load03_func=load03_func,
    load04_func=load04_func
)

cut_out_shot.cut_out_shot(
    rawdata_dir_name=rawdata_dir_name, 
    start_displacement=start_displacement, 
    end_displacement=end_displacement,
    start_sequential_number=start_sequential_number,
    end_sequential_number=end_sequential_number
)

2021-02-26 15:36:40,338 [INFO] Cut out shot start.
2021-02-26 15:36:40,570 [INFO] delete index 'shots-20201201010000-data' finished. result: {'acknowledged': True}
2021-02-26 15:36:40,770 [INFO] create index 'shots-20201201010000-data' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'shots-20201201010000-data'}
2021-02-26 15:36:40,830 [INFO] delete index 'shots-20201201010000-meta' finished. result: {'acknowledged': True}
2021-02-26 15:36:41,035 [INFO] create index 'shots-20201201010000-meta' finished. result: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'shots-20201201010000-meta'}
2021-02-26 15:37:08,879 [INFO] 436180 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20201201010000/tmp000.pkl.
2021-02-26 15:37:31,596 [INFO] Thoughput: 197795 doc/sec, processed_count: 10000000
2021-02-26 15:37:37,612 [INFO] 436180 data cut out from /home/ymiyamoto5/h-one-experimental-system/shared/data/20201201010000/tmp001.pkl.
202